# Modèle 1: CNN Custom avec Explainability

**Architecture CNN personnalisée avec:**
- 4 blocs convolutionnels (3→64→128→256→512 canaux)
- Batch Normalization pour stabilité
- Dropout pour régularisation
- Global Average Pooling
- Grad-CAM pour l'explicabilité

**Dataset:** 2200 images (11 classes d'épices)  
**Split:** 70% train / 15% val / 15% test

In [ ]:
# ====================================
# IMPORTS
# ====================================
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import cv2
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Configuration device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   CUDA device: {torch.cuda.get_device_name(0)}")

🖥️  Device: cpu
   PyTorch version: 2.10.0+cpu
   CUDA available: False


## 1. Dataset et DataLoader

In [ ]:
# ====================================
# DATASET CLASS
# ====================================
class SpiceDataset(Dataset):
    """
    Dataset personnalisé pour les épices.
    Charge les images depuis la structure: root_dir/class_name/*.jpg
    """
    def __init__(self, root_dir, transform=None):
        self.root_dir = Path(root_dir)
        self.transform = transform
        self.images = []
        self.labels = []
        
        # Récupérer les noms de classes (dossiers)
        self.class_names = sorted([d.name for d in self.root_dir.iterdir() if d.is_dir()])
        self.class_to_idx = {name: idx for idx, name in enumerate(self.class_names)}
        
        # Charger tous les chemins d'images
        for class_name in self.class_names:
            class_dir = self.root_dir / class_name
            for img_path in class_dir.glob('*.jpg'):
                self.images.append(img_path)
                self.labels.append(self.class_to_idx[class_name])
        
        print(f"   ✅ Loaded {len(self.images)} images from {root_dir}")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Charger l'image
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        
        # Appliquer les transformations
        if self.transform:
            image = self.transform(image)
        
        return image, label

print("✅ Dataset class définie")

✅ Dataset class définie


In [ ]:
# ====================================
# TRANSFORMATIONS (AUGMENTATION)
# ====================================
print("\n📋 Configuration des transformations d'images...\n")

# Train: Augmentation aggressive
train_transform = transforms.Compose([
    # 1. Géométriques (sur PIL Image)
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.RandomRotation(degrees=15),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    
    # 2. Couleur/Lumière (sur PIL Image)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.3),
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.5)),
    
    # 3. Conversion PIL → Tensor (CRUCIAL!)
    transforms.ToTensor(),
    
    # 4. Augmentation sur Tensor
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.1)),
    
    # 5. Normalisation ImageNet
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Val/Test: Pas d'augmentation
val_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("   ✅ Train transform: 10 augmentations")
print("   ✅ Val/Test transform: Normalisation seulement")


📋 Configuration des transformations d'images...

   ✅ Train transform: 10 augmentations
   ✅ Val/Test transform: Normalisation seulement


In [ ]:
# ====================================
# CRÉATION DES DATASETS ET DATALOADERS
# ====================================
print("\n📂 Chargement des datasets...\n")

# Créer les datasets
train_dataset = SpiceDataset('../dataset/splits/train', transform=train_transform)
val_dataset = SpiceDataset('../dataset/splits/val', transform=val_transform)
test_dataset = SpiceDataset('../dataset/splits/test', transform=val_transform)

print(f"\n   📊 Classes ({len(train_dataset.class_names)}): {train_dataset.class_names}")
print(f"   📊 Train: {len(train_dataset)} images")
print(f"   📊 Val: {len(val_dataset)} images")
print(f"   📊 Test: {len(test_dataset)} images")

# Créer les data loaders
batch_size = 32
num_workers = 2 
# Only use pin_memory if a GPU is available
pin_memory = torch.cuda.is_available()

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                         num_workers=num_workers, pin_memory=pin_memory)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, 
                       num_workers=num_workers, pin_memory=pin_memory)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, 
                        num_workers=num_workers, pin_memory=pin_memory)

print(f"\n   ✅ Batch size: {batch_size}")
print(f"   ✅ Num workers: {num_workers} (for faster data loading)")
print(f"   ✅ Pin memory: {pin_memory} (speeds up CPU-to-GPU transfer)")
print(f"   ✅ Train batches: {len(train_loader)}")
print(f"   ✅ Val batches: {len(val_loader)}")
print(f"   ✅ Test batches: {len(test_loader)}")

# Vérification du format de sortie
print("\n🔍 Vérification du format des données...")
sample_batch, sample_labels = next(iter(train_loader))
print(f"   ✅ Batch shape: {sample_batch.shape} (batch, channels, height, width)")
print(f"   ✅ Batch dtype: {sample_batch.dtype}")
print(f"   ✅ Batch range: [{sample_batch.min():.3f}, {sample_batch.max():.3f}]")
print(f"   ✅ Labels shape: {sample_labels.shape}")
assert sample_batch.shape[1:] == (3, 224, 224), "Shape incorrecte!"
print("\n✅✅✅ Format des données VALIDÉ!\n")


📂 Chargement des datasets...

   ✅ Loaded 1540 images from ../dataset/splits/train
   ✅ Loaded 330 images from ../dataset/splits/val
   ✅ Loaded 330 images from ../dataset/splits/test

   📊 Classes (11): ['anis', 'cannelle', 'carvi', 'clou_girofle', 'cubebe', 'cumin', 'curcuma', 'gingembre', 'paprika', 'poivre noir', 'safran']
   📊 Train: 1540 images
   📊 Val: 330 images
   📊 Test: 330 images

   ✅ Batch size: 32
   ✅ Num workers: 2 (for faster data loading)
   ✅ Train batches: 49
   ✅ Val batches: 11
   ✅ Test batches: 11

🔍 Vérification du format des données...


c:\Users\zaineb\.conda\envs\spices_env\lib\site-packages\torch\utils\data\dataloader.py:1118: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


## 2. Architecture CNN Custom

In [ ]:
# ====================================
# ARCHITECTURE CNN CUSTOM
# ====================================
class CustomCNN(nn.Module):
    """
    CNN Custom à 4 blocs avec architecture progressive:
    - Block 1: 3 → 64 canaux (détection features simples)
    - Block 2: 64 → 128 canaux (combinaison features)
    - Block 3: 128 → 256 canaux (patterns locaux)
    - Block 4: 256 → 512 canaux (patterns globaux)
    - Global Average Pooling
    - Classifier: 512 → 256 → 11 classes
    """
    def __init__(self, num_classes=11):
        super(CustomCNN, self).__init__()
        
        # Block 1: 3 → 64 (détection arêtes, couleurs)
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 224×224 → 112×112
        )
        
        # Block 2: 64 → 128 (formes locales)
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 112×112 → 56×56
        )
        
        # Block 3: 128 → 256 (objets locaux)
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 56×56 → 28×28
        )
        
        # Block 4: 256 → 512 (structure globale)
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)  # 28×28 → 14×14
        )
        
        # Global Average Pooling: 14×14×512 → 512
        self.gap = nn.AdaptiveAvgPool2d(1)
        
        # Classifier avec régularisation forte
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
        # Variables pour Grad-CAM
        self.gradients = None
        self.activations = None
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Sauvegarder activations pour Grad-CAM
        if x.requires_grad:
            x.register_hook(self.save_gradient)
        self.activations = x
        
        # Global Average Pooling + Classifier
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def save_gradient(self, grad):
        """Hook pour sauvegarder les gradients (Grad-CAM)"""
        self.gradients = grad


# Créer le modèle
print("\n🏗️  Création du modèle CNN Custom...\n")
model = CustomCNN(num_classes=11).to(device)

# Compter les paramètres
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(model)
print(f"\n📊 Statistiques du modèle:")
print(f"   Total paramètres: {total_params:,}")
print(f"   Paramètres entraînables: {trainable_params:,}")
print(f"   Taille estimée: ~{total_params * 4 / 1024 / 1024:.1f} MB (float32)")

## 3. Entraînement du Modèle

In [ ]:
# ====================================
# CONFIGURATION ENTRAÎNEMENT
# ====================================
print("\n⚙️  Configuration de l'entraînement...\n")

# Loss, Optimizer, Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', patience=3, factor=0.5
)

# OPTIMIZATION: Add GradScaler for Mixed Precision
scaler = GradScaler()

# Hyperparamètres
num_epochs = 30
best_val_acc = 0
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print(f"   ✅ Loss: CrossEntropyLoss")
print(f"   ✅ Optimizer: Adam (lr=0.001)")
print(f"   ✅ Scheduler: ReduceLROnPlateau (patience=3, factor=0.5)")
print(f"   ✅ Mixed Precision: Enabled (via GradScaler)")
print(f"   ✅ Epochs: {num_epochs}")
print(f"   ✅ Best model will be saved to: model_cnn_custom_best.pth\n")

In [ ]:
# ====================================
# FONCTIONS D'ENTRAÎNEMENT
# ====================================
from torch.cuda.amp import autocast, GradScaler

def train_epoch(model, loader, criterion, optimizer, scaler, device):
    """Entraîner le modèle sur une epoch avec Mixed Precision"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    pbar = tqdm(loader, desc='Training', leave=False)
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass with autocast
        with autocast():
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # Backward pass with scaler
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Métriques
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        # Update progress bar
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{100. * correct / total:.2f}%'
        })
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


def validate(model, loader, criterion, device):
    """Évaluer le modèle sur validation/test set avec Mixed Precision"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc='Validation', leave=False):
            images = images.to(device)
            labels = labels.to(device)
            
            # Forward pass with autocast
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            
            # Métriques
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    epoch_loss = running_loss / len(loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc


print("✅ Fonctions d'entraînement définies (avec Mixed Precision)")

In [ ]:
# ====================================
# BOUCLE D'ENTRAÎNEMENT
# ====================================
print("\n" + "="*60)
print("🔥 DÉBUT DE L'ENTRAÎNEMENT")
print("="*60 + "\n")

for epoch in range(num_epochs):
    print(f"\n{'='*60}")
    print(f"📍 Epoch {epoch+1}/{num_epochs}")
    print(f"{'='*60}")
    
    # Train (pass scaler)
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, scaler, device)
    
    # Validation
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    # Sauvegarder l'historique
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Afficher les résultats
    print(f"\n📊 Résultats Epoch {epoch+1}:")
    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"   Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    
    # Learning rate scheduler
    scheduler.step(val_loss)
    current_lr = optimizer.param_groups[0]['lr']
    print(f"   Learning Rate: {current_lr:.6f}")
    
    # Sauvegarder le meilleur modèle
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
            'val_loss': val_loss,
            'train_acc': train_acc,
            'train_loss': train_loss,
            'class_names': train_dataset.class_names
        }, '../models/model_cnn_custom_best.pth')
        print(f"   ✅ Nouveau meilleur modèle sauvegardé! Val Acc: {val_acc:.2f}%")

print("\n" + "="*60)
print(f"✅ ENTRAÎNEMENT TERMINÉ!")
print(f"   Meilleure Val Accuracy: {best_val_acc:.2f}%")
print("="*60 + "\n")

## 4. Visualisation des Résultats

In [ ]:
# ====================================
# VISUALISATION LOSS & ACCURACY
# ====================================
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2, marker='o', markersize=4)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2, marker='s', markersize=4)
axes[0].set_title('Loss Evolution', fontsize=16, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(alpha=0.3, linestyle='--')

# Accuracy plot
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2, marker='o', markersize=4)
axes[1].plot(history['val_acc'], label='Val Accuracy', linewidth=2, marker='s', markersize=4)
axes[1].set_title('Accuracy Evolution', fontsize=16, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy (%)', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(alpha=0.3, linestyle='--')
axes[1].axhline(y=best_val_acc, color='red', linestyle=':', label=f'Best: {best_val_acc:.2f}%', alpha=0.7)

plt.tight_layout()
plt.savefig('../models/training_curves_cnn_custom.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Graphiques sauvegardés: ../models/training_curves_cnn_custom.png")

## 5. Évaluation sur le Test Set

In [ ]:
# ====================================
# ÉVALUATION SUR TEST SET
# ====================================
print("\n📊 Évaluation sur le Test Set...\n")

# Charger le meilleur modèle
checkpoint = torch.load('model_cnn_custom_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✅ Meilleur modèle chargé (Epoch {checkpoint['epoch']}, Val Acc: {checkpoint['val_acc']:.2f}%)\n")

# Évaluation
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc='Testing'):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        _, predicted = outputs.max(1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Accuracy
test_acc = 100. * np.sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)
print(f"\n🎯 Test Accuracy: {test_acc:.2f}%\n")

# Classification Report
print("="*70)
print("📋 CLASSIFICATION REPORT")
print("="*70)
print(classification_report(
    all_labels, all_preds, 
    target_names=train_dataset.class_names,
    digits=4
))

In [ ]:
# ====================================
# MATRICE DE CONFUSION
# ====================================
cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(12, 10))
sns.heatmap(
    cm, annot=True, fmt='d', cmap='Blues',
    xticklabels=train_dataset.class_names,
    yticklabels=train_dataset.class_names,
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - CNN Custom', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('True Label', fontsize=13)
plt.xlabel('Predicted Label', fontsize=13)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig('../models/confusion_matrix_cnn_custom.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Matrice de confusion sauvegardée: ../models/confusion_matrix_cnn_custom.png")

## 6. Explainability: Grad-CAM

Grad-CAM (Gradient-weighted Class Activation Mapping) permet de visualiser quelles régions de l'image sont importantes pour la prédiction du modèle.

In [ ]:
# ====================================
# GRAD-CAM IMPLEMENTATION
# ====================================
def generate_gradcam(model, image, target_class):
    """
    Génère une heatmap Grad-CAM pour une image et une classe cible.
    
    Args:
        model: Le modèle CNN
        image: Image tensor (C, H, W)
        target_class: Classe cible (int)
    
    Returns:
        cam: Heatmap Grad-CAM (H, W)
    """
    model.eval()
    image = image.unsqueeze(0).to(device)
    image.requires_grad = True
    
    # Forward pass
    output = model(image)
    
    # Backward pass sur la classe cible
    model.zero_grad()
    output[0, target_class].backward()
    
    # Récupérer gradients et activations
    gradients = model.gradients.cpu().data.numpy()[0]  # (512, 14, 14)
    activations = model.activations.cpu().data.numpy()[0]  # (512, 14, 14)
    
    # Calculer les poids (global average pooling des gradients)
    weights = np.mean(gradients, axis=(1, 2))  # (512,)
    
    # Calculer la CAM (weighted sum des activations)
    cam = np.zeros(activations.shape[1:], dtype=np.float32)  # (14, 14)
    for i, w in enumerate(weights):
        cam += w * activations[i]
    
    # ReLU (garder seulement les contributions positives)
    cam = np.maximum(cam, 0)
    
    # Normaliser entre 0 et 1
    if cam.max() > 0:
        cam = cam / cam.max()
    
    # Redimensionner à la taille de l'image originale
    cam = cv2.resize(cam, (224, 224))
    
    return cam


def denormalize_image(tensor):
    """Dénormalise un tensor d'image (inverse ImageNet normalization)"""
    img = tensor.cpu().numpy().transpose(1, 2, 0)
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    return img


def show_gradcam(image, cam, title, save_path=None):
    """
    Affiche l'image originale, la heatmap Grad-CAM, et la superposition.
    
    Args:
        image: Image tensor (C, H, W)
        cam: Heatmap Grad-CAM (H, W)
        title: Titre pour la visualisation
        save_path: Chemin pour sauvegarder (optionnel)
    """
    # Dénormaliser l'image
    img = denormalize_image(image)
    
    # Créer la heatmap colorée
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) / 255.0
    
    # Superposer (60% image originale + 40% heatmap)
    overlay = 0.6 * img + 0.4 * heatmap
    overlay = np.clip(overlay, 0, 1)
    
    # Afficher
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    axes[0].imshow(img)
    axes[0].set_title('Image Originale', fontsize=13, fontweight='bold')
    axes[0].axis('off')
    
    axes[1].imshow(cam, cmap='jet')
    axes[1].set_title('Grad-CAM Heatmap', fontsize=13, fontweight='bold')
    axes[1].axis('off')
    
    axes[2].imshow(overlay)
    axes[2].set_title(f'Superposition\n{title}', fontsize=13, fontweight='bold')
    axes[2].axis('off')
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    
    plt.show()


print("✅ Fonctions Grad-CAM définies")

In [ ]:
# ====================================
# VISUALISATION GRAD-CAM
# ====================================
print("\n🎨 Génération des visualisations Grad-CAM...\n")

# Récupérer quelques exemples du test set
model.eval()
dataiter = iter(test_loader)
images, labels = next(dataiter)

num_examples = min(5, len(images))

for i in range(num_examples):
    image = images[i]
    true_label = labels[i].item()
    
    # Prédiction
    with torch.no_grad():
        output = model(image.unsqueeze(0).to(device))
        pred_label = output.argmax(1).item()
        confidence = torch.softmax(output, dim=1)[0, pred_label].item()
    
    # Générer Grad-CAM sur la classe prédite
    cam = generate_gradcam(model, image, pred_label)
    
    # Créer le titre
    pred_class = train_dataset.class_names[pred_label]
    true_class = train_dataset.class_names[true_label]
    is_correct = "✅" if pred_label == true_label else "❌"
    title = f"{is_correct} Pred: {pred_class} ({confidence*100:.1f}%) | True: {true_class}"
    
    # Afficher
    show_gradcam(image, cam, title, save_path=f'../models/gradcam_example_{i+1}.png')
    
    print(f"   {i+1}. {title}")

print(f"\n✅ {num_examples} visualisations Grad-CAM générées et sauvegardées")

## 7. Sauvegarde des Résultats

In [ ]:
# ====================================
# SAUVEGARDE DES RÉSULTATS
# ====================================
print("\n💾 Sauvegarde des résultats finaux...\n")

results = {
    'model': 'CNN Custom',
    'architecture': {
        'blocks': 4,
        'channels': '3→64→128→256→512',
        'total_params': int(total_params),
        'trainable_params': int(trainable_params)
    },
    'training': {
        'epochs': num_epochs,
        'batch_size': batch_size,
        'optimizer': 'Adam',
        'learning_rate': 0.001,
        'scheduler': 'ReduceLROnPlateau'
    },
    'performance': {
        'best_val_acc': float(best_val_acc),
        'test_acc': float(test_acc),
        'best_epoch': int(checkpoint['epoch'])
    },
    'dataset': {
        'num_classes': len(train_dataset.class_names),
        'class_names': train_dataset.class_names,
        'train_size': len(train_dataset),
        'val_size': len(val_dataset),
        'test_size': len(test_dataset)
    },
    'history': {
        'train_loss': [float(x) for x in history['train_loss']],
        'train_acc': [float(x) for x in history['train_acc']],
        'val_loss': [float(x) for x in history['val_loss']],
        'val_acc': [float(x) for x in history['val_acc']]
    }
}

# Sauvegarder en JSON
with open('../models/results_cnn_custom.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print("✅ Résultats sauvegardés dans: ../models/results_cnn_custom.json")
print("\n📊 Résumé Final:")
print("="*60)
print(f"   Modèle: CNN Custom ({total_params:,} paramètres)")
print(f"   Meilleure Val Accuracy: {best_val_acc:.2f}% (Epoch {checkpoint['epoch']})")
print(f"   Test Accuracy: {test_acc:.2f}%")
print(f"   Classes: {len(train_dataset.class_names)}")
print("="*60)
print("\n✅ Tous les résultats ont été sauvegardés avec succès!")

# Modèle 1: CNN Custom avec Explainability

Architecture CNN personnalisée avec:
- Plusieurs couches convolutionnelles
- Batch Normalization
- Dropout pour régularisation
- Grad-CAM pour l'explicabilité

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from pathlib import Path
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import json
import cv2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Device: {device}")

## 1. Dataset et DataLoader

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224), interpolation=transforms.InterpolationMode.LANCZOS),
    
    # Geometric transforms (work on PIL Image)
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    
    # Color/Light transforms (work on PIL Image)
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAutocontrast(p=0.3),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.3),
    
    # GaussianBlur works on PIL
    transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 0.5)),
    
    # ✅ CONVERT TO TENSOR FIRST (before RandomErasing!)
    transforms.ToTensor(),
    
    # Noise/Texture transforms (MUST work on Tensor)
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.1)),
    
    # ImageNet normalization (works on Tensor)
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

## 2. Architecture CNN Custom

In [ ]:
class CustomCNN(nn.Module):
    def __init__(self, num_classes=11):
        super(CustomCNN, self).__init__()
        
        # Block 1: 3 -> 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 224 -> 112
        )
        
        # Block 2: 64 -> 128
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 112 -> 56
        )
        
        # Block 3: 128 -> 256
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 56 -> 28
        )
        
        # Block 4: 256 -> 512
        self.conv4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)  # 28 -> 14
        )
        
        # Global Average Pooling + Classifier
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )
        
        # Pour Grad-CAM
        self.gradients = None
        self.activations = None
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        # Sauvegarder pour Grad-CAM
        if x.requires_grad:
            x.register_hook(self.save_gradient)
        self.activations = x
        
        x = self.gap(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def save_gradient(self, grad):
        self.gradients = grad

# Créer le modèle
model = CustomCNN(num_classes=11).to(device)
print("\n🏗️  Architecture du modèle:")
print(model)

# Compter les paramètres
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n📊 Total paramètres: {total_params:,}")
print(f"📊 Paramètres entraînables: {trainable_params:,}")

## 3. Entraînement

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5)

def train_epoch(model, loader, criterion, optimizer, device):
    """Entraîner une epoch avec vérification des tensors"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in tqdm(loader, desc='Training'):
        # Vérifier que ce sont des tensors
        if not isinstance(images, torch.Tensor):
            raise TypeError(f"Expected tensor, got {type(images)}")
        
        # Déplacer vers device
        images = images.to(device)
        labels = labels.to(device)
        
        # Vérifier la shape
        assert images.dim() == 4, f"Expected 4D, got {images.dim()}D: {images.shape}"
        assert images.shape[1:] == (3, 224, 224), f"Wrong shape: {images.shape}"
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), 100. * correct / total

def validate(model, loader, criterion, device):
    """Valider avec vérification des tensors"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            # Vérifier que ce sont des tensors
            if not isinstance(images, torch.Tensor):
                raise TypeError(f"Expected tensor, got {type(images)}")
            
            # Déplacer vers device
            images = images.to(device)
            labels = labels.to(device)
            
            # Vérifier la shape
            assert images.dim() == 4, f"Expected 4D, got {images.dim()}D: {images.shape}"
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), 100. * correct / total

In [ ]:

# =============================================
# FINAL CHECK before training
# =============================================
print("🎯 FINAL VERIFICATION before training...\n")

# Test single sample from each loader
loaders_to_test = [
    ("Train", train_loader),
    ("Validation", val_loader),
    ("Test", test_loader)
]

for loader_name, loader in loaders_to_test:
    images, labels = next(iter(loader))
    
    # Verify it's a tensor
    assert isinstance(images, torch.Tensor), f"{loader_name}: Not a tensor!"
    assert images.dim() == 4, f"{loader_name}: Wrong dimensions!"
    assert images.shape[1:] == (3, 224, 224), f"{loader_name}: Wrong shape!"
    
    print(f"✅ {loader_name:12} | Shape: {images.shape} | Type: {type(images).__name__} | Range: [{images.min():.3f}, {images.max():.3f}]")

print("\n✅ All loaders verified and ready for training!\n")

In [ ]:
# Entraîner le modèle
num_epochs = 30
best_val_acc = 0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

for epoch in range(num_epochs):
    print(f"\n{'='*50}")
    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"{'='*50}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
    
    scheduler.step(val_loss)
    
    # Sauvegarder le meilleur modèle
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_acc': val_acc,
        }, 'model_cnn_custom_best.pth')
        print(f"✅ Meilleur modèle sauvegardé! Val Acc: {val_acc:.2f}%")

## 4. Visualisation des Résultats

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_title('Loss Evolution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy
axes[1].plot(history['train_acc'], label='Train Accuracy', linewidth=2)
axes[1].plot(history['val_acc'], label='Val Accuracy', linewidth=2)
axes[1].set_title('Accuracy Evolution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Évaluation sur le Test Set

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Charger le meilleur modèle
checkpoint = torch.load('model_cnn_custom_best.pth')
model.load_state_dict(checkpoint['model_state_dict'])

# Évaluer
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = outputs.max(1)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Classification report
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, target_names=train_dataset.class_names))

# Confusion matrix
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=train_dataset.class_names,
            yticklabels=train_dataset.class_names)
plt.title('Confusion Matrix - CNN Custom', fontsize=14, fontweight='bold')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

## 6. Explainability: Grad-CAM

In [ ]:
def generate_gradcam(model, image, label):
    """Génère une heatmap Grad-CAM"""
    model.eval()
    image = image.unsqueeze(0).to(device)
    image.requires_grad = True
    
    # Forward pass
    output = model(image)
    
    # Backward pass
    model.zero_grad()
    output[0, label].backward()
    
    # Obtenir gradients et activations
    gradients = model.gradients.cpu().data.numpy()[0]
    activations = model.activations.cpu().data.numpy()[0]
    
    # Calculer les poids
    weights = np.mean(gradients, axis=(1, 2))
    
    # Calculer la CAM
    cam = np.zeros(activations.shape[1:], dtype=np.float32)
    for i, w in enumerate(weights):
        cam += w * activations[i]
    
    cam = np.maximum(cam, 0)
    cam = cam / cam.max() if cam.max() != 0 else cam
    cam = cv2.resize(cam, (224, 224))
    
    return cam

def show_gradcam(image, cam, title):
    """Affiche l'image avec la heatmap Grad-CAM"""
    # Dénormaliser l'image
    img = image.cpu().numpy().transpose(1, 2, 0)
    img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
    img = np.clip(img, 0, 1)
    
    # Créer heatmap
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB) / 255.0
    
    # Superposer
    overlay = 0.6 * img + 0.4 * heatmap
    overlay = np.clip(overlay, 0, 1)
    
    # Afficher
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    axes[0].imshow(img)
    axes[0].set_title('Image Originale')
    axes[0].axis('off')
    
    axes[1].imshow(cam, cmap='jet')
    axes[1].set_title('Grad-CAM Heatmap')
    axes[1].axis('off')
    
    axes[2].imshow(overlay)
    axes[2].set_title(f'Superposition - {title}')
    axes[2].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
import cv2

# Visualiser Grad-CAM sur quelques exemples
model.eval()
dataiter = iter(test_loader)
images, labels = next(dataiter)

for i in range(min(5, len(images))):
    image = images[i]
    label = labels[i].item()
    
    # Prédiction
    with torch.no_grad():
        output = model(image.unsqueeze(0).to(device))
        pred = output.argmax(1).item()
    
    # Générer Grad-CAM
    cam = generate_gradcam(model, image, pred)
    
    # Afficher
    title = f"Pred: {train_dataset.class_names[pred]} | True: {train_dataset.class_names[label]}"
    show_gradcam(image, cam, title)

## 7. Sauvegarde des Métriques

In [ ]:
# Sauvegarder l'historique et les métriques
results = {
    'model': 'CNN Custom',
    'best_val_acc': float(best_val_acc),
    'test_acc': float(100. * sum(np.array(all_preds) == np.array(all_labels)) / len(all_labels)),
    'num_params': int(total_params),
    'history': history
}

with open('results_cnn_custom.json', 'w') as f:
    json.dump(results, f, indent=2)

print("\n✅ Résultats sauvegardés dans results_cnn_custom.json")
print(f"📊 Test Accuracy: {results['test_acc']:.2f}%")